In [3]:
import numpy as np
#from imageio import imread, imwrite
import cv2

## Preparation


In [98]:
grainger_image = cv2.imread('grainger.jpg').astype(np.uint8)

## Naive Method: Increase blue channel

In [4]:
arr = grainger_image * np.array([0.5, 0.2, 0.1])
arr2 = (255 * arr/arr.max()).astype(np.uint8)

gamma = 2
gamma_img = np.array(255 * (arr2/255) ** gamma, dtype="uint8")
#window_name = 'image'
#cv2.imshow(window_name, gamma_img)
cv2.imwrite("night_final.png", gamma_img)

True

## Intermediate Method

In [177]:
def turn_day_to_night(image):
    '''
    Intermediate method looks to increase the contrast of an image
    Decrease brightness
    Decrease saturation
    Increase blue-tint of image
    Add noise to image
    Increase contrast
    '''
    image_width, image_height, color_channels = image.shape

    # Convert image to YUV color space and decrease brightness by half
    yuv_image = cv2.cvtColor(image, cv2.COLOR_BGR2YUV)
    yuv_image[:,:,0] = np.clip(yuv_image[:,:,0] * 0.5, 0, 255) 
    bgr_image = cv2.cvtColor(yuv_image, cv2.COLOR_YUV2BGR)

    
    # Convert image to HSV color space and decrease warmth
    bgr_image[:,:,0] = bgr_image[:,:,0] + 30
    hsv_image = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    hsv_image[:, :, 1] = hsv_image[:, :, 1] * 0.8
    bgr_image = cv2.cvtColor(hsv_image, cv2.COLOR_HSV2BGR)
    
    # Adjust contrast
    bgr_image = cv2.convertScaleAbs(bgr_image, alpha=0.8, beta=0)

    #Adding noise to image
    noise_filter = np.zeros((image_width, image_height, color_channels), np.uint8)
    cv2.randu(noise_filter,(0),(10))
    
    result_image = bgr_image + noise_filter

    return result_image

In [178]:
night_image = turn_day_to_night(grainger_image)
cv2.imwrite("night_final.png", night_image)

True

In [191]:
def super_turn_day_to_night(image, alpha):
    image_width, image_height, color_channels = image.shape
    # Convert image to XYZ color space
    xyz_image = cv2.cvtColor(image, cv2.COLOR_BGR2XYZ)

    # For the Scotopic Luminance of the image
    x = xyz_image[:,:,0]
    y = xyz_image[:,:,1]
    z = xyz_image[:,:,2]
    scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)
    scotopic_luminance = np.clip(np.nan_to_num(scotopic_luminance), 0, 255).astype(np.uint8)

    # Create a
    scotopic_lum_img = np.zeros((image_width, image_height, color_channels)).astype(np.uint8)
    blue = np.array([.9, .5, .5]).astype(np.float64)
    k = 5
    for c in range(color_channels):
        scotopic_lum_img[:,:,c] = scotopic_luminance * blue[c] * k

    # Blend original and scotopic images pixel by pixel
    result_image = image * (1 - alpha) + scotopic_lum_img * alpha
    return result_image
    

In [192]:
night_image = super_turn_day_to_night(grainger_image, .8)
cv2.imwrite("night_final.png", night_image)
print(night_image)

C:\Users\uchei\AppData\Local\Temp\ipykernel_29376\1014319905.py:10: RuntimeWarning: divide by zero encountered in divide
  scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)
C:\Users\uchei\AppData\Local\Temp\ipykernel_29376\1014319905.py:10: RuntimeWarning: invalid value encountered in divide
  scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)
C:\Users\uchei\AppData\Local\Temp\ipykernel_29376\1014319905.py:10: RuntimeWarning: invalid value encountered in multiply
  scotopic_luminance = y * (1.33 * (1 + (y + z)/x) - 1.68)


[[[ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  ...
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]]

 [[ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  ...
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]]

 [[ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  [ 74.4 105.2 104. ]
  ...
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]
  [147.6 149.2 149.2]]

 ...

 [[130.2 222.8 221. ]
  [118.8 216.2 214.4]
  [166.  127.4 125.6]
  ...
  [115.6 214.4 211.8]
  [ 27.2 164.8 162.4]
  [ 77.6  77.8  75. ]]

 [[111.4 212.  210.2]
  [139.8 112.4 110.6]
  [173.4 131.6 129.8]
  ...
  [210.  152.2 149. ]
  [180.4 135.8 133. ]
  [151.  119.2 116. ]]

 [[137.6  22.2  20.4]
  [132.4 108.2 106.4]
  [111.4 212.  210.2]
  ...
  [117.6  99.8  96.6]
  [117.6  99.8  96.6]
  [ 12.2 156.4 153.2]]]


Sources used:
https://stackoverflow.com/questions/72888224/image-looks-overexposed-nearly-all-white-when-using-np-zeros-with-opencv-imsho